## Use Pretrained model to improve accuracy

In [60]:
from keras.datasets import cifar10
from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten,Dropout,BatchNormalization,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.applications import VGG16,InceptionV3
from keras.models import Model
import scipy as sp
import numpy as np
from PIL import Image
epochs = 20
batch_size = 64
num_classes = 10

In [40]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [41]:
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [42]:
X_train,X_val,Y_train,Y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=11)

In [43]:
a = X_train[0]
np.array(Image.fromarray(a).resize((42,42)))

array([[[126, 133, 149],
        [124, 131, 147],
        [124, 131, 147],
        ...,
        [127, 134, 150],
        [127, 134, 150],
        [123, 130, 146]],

       [[128, 136, 152],
        [127, 134, 150],
        [127, 134, 150],
        ...,
        [129, 136, 152],
        [129, 136, 152],
        [126, 133, 149]],

       [[128, 136, 152],
        [127, 134, 150],
        [127, 134, 150],
        ...,
        [129, 136, 152],
        [129, 136, 152],
        [126, 133, 149]],

       ...,

       [[ 91, 117,  92],
        [ 91, 114,  88],
        [ 91, 114,  88],
        ...,
        [ 80, 106,  85],
        [ 80, 106,  85],
        [ 84, 103,  85]],

       [[ 91, 117,  92],
        [ 91, 114,  88],
        [ 91, 114,  88],
        ...,
        [ 80, 106,  85],
        [ 80, 106,  85],
        [ 84, 103,  85]],

       [[ 86, 118,  93],
        [ 88, 113,  86],
        [ 88, 113,  86],
        ...,
        [ 80, 104,  83],
        [ 80, 104,  83],
        [ 80, 103,  82]]

In [44]:
X_train = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in X_train])
X_val = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in X_val])
X_test = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in x_test])

In [45]:
base = VGG16(weights='imagenet',
                    include_top=False,
                    input_shape = (64,64,3))


In [90]:
base.summary()
base.layers.pop()
inputs = base.layers[-3].output

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [103]:
base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [91]:
for layer in base.layers:
     layer.trainable = False

In [97]:
'''x = Flatten()(inputs)
x = BatchNormalization()(x)
x = Dense(256,activation='relu')(x)'''
x = Conv2D(512,(3,3),activation='relu')(inputs)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
x = Dense(num_classes,activation='softmax')(x)
model = Model(base.input,x)

In [98]:
model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0  

In [99]:
model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics=['acc'],
             )

In [100]:
model_save_path = "val_loss-{val_loss:.4f}-val_acc{val_acc:4f}.hdf5"
callbacks_list = [
    EarlyStopping(
        monitor='val_acc',
        patience=2
    ),
    ModelCheckpoint(
        model_save_path,
        monitor='val_acc',
        save_best_only = True
    )
    
]

In [101]:
history = model.fit(X_train,Y_train,
         batch_size = batch_size,
         epochs = epochs,
         validation_data = (X_val,Y_val),
         callbacks = callbacks_list)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 11s 271us/step - loss: 0.8270 - acc: 0.7370 - val_loss: 0.5449 - val_acc: 0.8102
Epoch 2/20
40000/40000 [==============================] - 11s 265us/step - loss: 0.5273 - acc: 0.8202 - val_loss: 0.4725 - val_acc: 0.8348
Epoch 3/20
40000/40000 [==============================] - 11s 266us/step - loss: 0.4497 - acc: 0.8440 - val_loss: 0.4123 - val_acc: 0.8570
Epoch 4/20
40000/40000 [==============================] - 11s 265us/step - loss: 0.4062 - acc: 0.8588 - val_loss: 0.3873 - val_acc: 0.8658
Epoch 5/20
40000/40000 [==============================] - 11s 266us/step - loss: 0.3671 - acc: 0.8712 - val_loss: 0.3808 - val_acc: 0.8716
Epoch 6/20
40000/40000 [==============================] - 11s 266us/step - loss: 0.3480 - acc: 0.8789 - val_loss: 0.3855 - val_acc: 0.8702
Epoch 7/20
40000/40000 [==============================] - 11s 268us/step - loss: 0.3252 - acc: 0.8852 - val_loss: 0.

In [102]:
tt = load_model('val_loss-0.3430-val_acc0.888000.hdf5')
scores = tt.evaluate(X_test,y_test,verbose=1)
scores

10000/10000 [==============================] - 2s 217us/step


[0.351914974462986, 0.8827999830245972]

## VGG16 = 0.823, img_size = 48,48
## VGG16 = 0.854, img_size = 64,64
## VGG16 = 0.882, img_size = 64,64 --> with GAP
